<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/CSC645/blob/master/6zMixedData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
!git clone https://github.com/emanhamed/Houses-dataset


In [ ]:
import pandas as pd
df=pd.read_csv("Houses-dataset/Houses Dataset/HousesInfo.txt",header=None,delim_whitespace=True,
               names=["bedrooms","bathrooms","size","zipcode","price"])

In [ ]:
df[0:10]

count the number of houses per zipcode

In [ ]:
zipcodes=df['zipcode'].value_counts().keys().tolist()
counts=df['zipcode'].value_counts().tolist()

remove all the entries with zipcodes containing less than 20 houses

In [ ]:
for count,zipcode in zip(counts,zipcodes):
  if count<20:
    idx=df[df['zipcode']==zipcode].index
    df.drop(idx,inplace=True)

In [ ]:
df[0:10]

In [ ]:
import cv2

image_list=[]
prefix="Houses-dataset/Houses Dataset/"
suffix="_frontal.jpg"
for idx in df.index.tolist():
  path=prefix+str(idx)+suffix
  img=cv2.imread(path)
  img=cv2.resize(img,(48,48))
  image_list.append(img)

In [ ]:

import numpy as np
images=np.stack(image_list)

In [ ]:
images.shape

In [ ]:
images=images/255.

In [ ]:
values=df.values
prices=values[:,4]
features=values[:,0:4]

In [ ]:
print(images.shape,values.shape)

In [ ]:
train_images=images[0:300]
test_images=images[300:384]
train_features=features[0:300]
test_features=features[300:384]
train_prices=prices[0:300]/100000.
test_prices=prices[300:384]/100000.

In [ ]:
from tensorflow.keras.layers import Dense,Input,concatenate,Flatten
from tensorflow.keras.models import Model


In [ ]:
features_input=Input(shape=(4,),name="features_input")
features_layers=Dense(64,activation="relu")(features_input)
features_layers=Dense(32,activation="relu")(features_layers)
features_layers=Dense(16,activation="relu")(features_layers)

features_output=Dense(1,activation="linear")(features_layers)
image_input=Input(shape=(48,48,3),name="image_input")
flatten=Flatten()(image_input)
image_layers=Dense(64,activation="relu")(flatten)
image_output=Dense(1,activation="linear")(image_layers)

features_model=Model(inputs=features_input,outputs=features_output)
image_model=Model(inputs=image_input,outputs=image_output)

In [ ]:
both=concatenate([image_output,features_output])

In [ ]:
both=Dense(10,activation='relu')(both)
both=Dense(1,activation='linear')(both)
model=Model(inputs=[features_input,image_input],outputs=both)


In [ ]:
tf.keras.utils.plot_model(model,show_shapes=True)

In [ ]:
opt=tf.keras.optimizers.RMSprop(lr=1e-3)
model.compile(optimizer=opt,loss='mean_absolute_percentage_error')
features_model.compile(optimizer="Adam",loss="mean_absolute_percentage_error")

In [ ]:
model.fit(x=[train_features,train_images],y=train_prices,epochs=1000)


In [ ]:
features_model.fit(x=train_features,y=train_prices,epochs=1000)

In [ ]:
model.evaluate([test_features,test_images],test_prices)
features_model.evaluate(test_features,test_prices)

In [ ]:
combined_predict=np.squeeze(model.predict([test_features[0:10],test_images[0:10]]))
features_predict=np.squeeze(features_model.predict(test_features[0:10]))

In [ ]:
print(np.abs(test_prices[0:10]-features_predict)/test_prices[0:10])
print(np.abs(test_prices[0:10]-combined_predict)/test_prices[0:10])